In [13]:
import spacy
import pandas as pd
from gensim.models import Word2Vec

In [14]:
nlp = spacy.load("ru_core_news_sm")

def prepare_content(content):
    prepared_words = nlp(content)

    words = [word.lemma_ for word in prepared_words if word.is_alpha and not word.is_stop]

    return words

In [15]:
prepared_data = []

with open('news.txt', 'r', encoding='utf-8') as file:
    for line in file:
        category, title_content = line.strip().split('\t', 1)
        prepared_data.append([category, title_content])
df = pd.DataFrame(prepared_data, columns=['category', 'content'])

df['processed_content'] = df['content'].apply(prepare_content)
df



,category,content,processed_content
0,style,Rolex наградит победителей регаты\tПарусная го...,"[rolex, наградить, победитель, регата, парусны..."
1,sport,Матс Сундин стал советником тренера сборной Шв...,"[матс, сундин, советник, тренер, сборная, швец..."
2,media,Брендом года по версии EFFIE впервые стал горо...,"[бренд, год, версия, effie, впервые, город, гр..."
3,economics,Цена нефти WTI снизилась после публикации данн...,"[цена, нефть, wti, снизиться, публикация, запа..."
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"[сбербанк, распродать, банкир, миллиардный, до..."
...,...,...,...
9995,economics,Бывший пропагандист Буша нашел работу в торгов...,"[бывший, пропагандист, буша, найти, работа, то..."
9996,travel,Россияне предпочли смартфон любимому человеку ...,"[россиянин, предпочесть, смартфон, любимый, че..."
9997,life,Потерявшаяся пять недель назад американка найд...,"[потеряться, пять, неделя, американка, найти, ..."
9998,media,"НТВ покажет решающие матчи ""Зенита"" и ЦСКА в Л...","[нтв, показать, решающий, матч, зенит, цска, л..."


Avarage Vector

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [17]:
model = Word2Vec(df['processed_content'], vector_size=100, window=5, min_count=1, workers=12)
model.save("word2vec.model")

In [18]:
word2vec_model = Word2Vec.load('word2vec.model')

def news_vector(word2vec_model, words):
    tokens = [word for word in words if word in word2vec_model.wv.key_to_index]
    if len(tokens) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[tokens], axis=0)

In [19]:
df['news_vector'] = df['processed_content'].apply(lambda x: news_vector(word2vec_model, x))

vectors = np.array(df['news_vector'].tolist())
labels = df['category']

vectors_train, vectors_test, labels_train, labels_test = train_test_split(vectors, labels, test_size=0.2)


In [20]:
svm = SVC()
svm.fit(vectors_train, labels_train)

labels_pred = svm.predict(vectors_test)
print(classification_report(labels_test, labels_pred))

              precision    recall  f1-score   support

    business       0.67      0.03      0.05        72
     culture       0.83      0.88      0.85       268
   economics       0.70      0.89      0.78       273
      forces       0.63      0.75      0.69       148
        life       0.69      0.83      0.75       242
       media       0.82      0.80      0.81       309
     science       0.85      0.72      0.78       306
       sport       0.97      0.97      0.97       308
       style       1.00      0.67      0.80        24
      travel       0.85      0.22      0.35        50

    accuracy                           0.79      2000
   macro avg       0.80      0.67      0.68      2000
weighted avg       0.80      0.79      0.78      2000



TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [22]:
df['tf_idf'] = df['processed_content'].apply(' '.join)

tf_idf_vector = TfidfVectorizer()

tf = tf_idf_vector.fit_transform(df['tf_idf'])

In [23]:
labels = df['category']
tf_train, tf_test, labels_train, labels_test = train_test_split(tf, labels, test_size=0.2)

In [24]:
tf_svm = LinearSVC()
tf_svm.fit(tf_train, labels_train)

labels_pred = tf_svm.predict(tf_test)
print(classification_report(labels_test, labels_pred))


              precision    recall  f1-score   support

    business       0.73      0.52      0.61        86
     culture       0.91      0.94      0.93       282
   economics       0.84      0.88      0.86       250
      forces       0.78      0.86      0.82       157
        life       0.83      0.85      0.84       250
       media       0.89      0.86      0.88       288
     science       0.91      0.87      0.89       282
       sport       0.96      0.96      0.96       334
       style       0.89      0.80      0.84        40
      travel       0.73      0.77      0.75        31

    accuracy                           0.88      2000
   macro avg       0.85      0.83      0.84      2000
weighted avg       0.87      0.88      0.87      2000

